# 受講者の演習環境バックアップをリストアします
<HR>

受講者の実行した演習環境バックアップファイルを展開して、指定された位置に配置します。

### 1. リストアの作業用フォルダを作成します

* リストアするバックアップファルを格納して、展開するための作業用フォルダです。
* フォルダを変更する場合は、RESTORE_WORK_DIR を変更して下さい。
* フォルダが既存の場合、フォルダ配下のファイルは削除されます。必要があれば事前に退避して下さい。

**次のセルを実行して、作業用フォルダを作成して下さい。**

In [ ]:
RESTORE_WORK_DIR="$HOME/restore"

# create restore work directory
[ -e $RESTORE_WORK_DIR ] || {
  sudo mkdir -p $RESTORE_WORK_DIR
  sudo chown $USER:$USER $RESTORE_WORK_DIR
}

# creanlup restore work directory
rm -rf $RESTORE_WORK_DIR/*

### 2. バックアップファイルを転送します

* リストアする バックアップファイル(tar.gz 形式)を、上記で作成した作業用フォルダに転送して下さい。
* バックアップファイルがローカルマシンにある場合は、下記の手順でアップロードできます。
  1. ブラウザのHome画面で、リストア作業用フォルダを開きます。
  2. 画面右上の[Upload]ボタンをクリックします。ファイル選択ダイアログが開きます。
  3. ファイルを選択して[開く]ボタンをクリックします。 バックアップファイル名がリストされます。
  4. バックアップファイル名の行の右端に表示される[Upload]ボタンをクリックします。

### 3. バックアップファイル名を指定します

* ARCHIVE_FILE_NAME にリストアするバックアップファイル名を指定して下さい。
```
ARCHIVE_FILE_NAME="course_backup.tar.gz"
```
* バックアップファイルは、上のセルで作成した RESTORE_WORK_DIR に格納されているものとします。

**次のセルでバックアップファイル名を指定して、セルを実行して下さい。**

In [ ]:
ARCHIVE_FILE_NAME=""

### 4. バックアップファイルの内容を確認します

* 次のセルを実行して、転送したバックアップファイルの内容を確認して下さい。
* バックアップしたファイル/フォルダをどこに配置するかの設定に利用します。

In [ ]:
ARCHIVE_FILE_PATH="$HOME/restore/$ARCHIVE_FILE_NAME"
tar tvfz $ARCHIVE_FILE_PATH

### 5. 展開したファイル/フォルダの配置先を指定します

* バックアップファイルはホームディレクトリ配下に展開されます。
* RESTORE_DICに、展開したファイル/フォルダを、ホームディレクトリのどこに配置するかを、「配置元  配置先」の形式で指定して下さい。
```
RESTORE_DIC=(
"配置するファイル/フォルダ   ホームディレクトリ配下の配置先"
)
```
例 : バックアップが下記の構成で
```
drwxr-xr-x ksasaki11b819/ksasaki11b819 0 2017-05-14 04:24 work/
-rw-r--r-- ksasaki11b819/ksasaki11b819 436 2017-05-14 04:24 work/Untitled.ipynb
-rw-r--r-- ksasaki11b819/ksasaki11b819 436 2017-05-14 04:24 work/Untitled1.ipynb
-rw-r--r-- ksasaki11b819/ksasaki11b819 436 2017-05-14 04:25 test.ipynb
```
 フォルダ「work」をホームディレクトリの「draft」へ、「test.ipynb」をホームディレクトリ直下へ配置する場合
```
RESTORE_DIC=(
"work draft"
"'*.ipynb' ."
)
```
  + 配置先にホームディレクトリ直下を指定する場合は、"."を指定して下さい。
  + 配置元に"*"を含む場合は、配置元をシングルクオートで括って下さい。

**次のセルで配置先を設定して、セルを実行して下さい。**

In [ ]:
RESTORE_DIC=(
)

### 6. 次のセルを実行して、バックアップファイルを展開して下さい


バックアップファイルを展開して、RESTORE_DIC で指定した位置に配置します。

In [ ]:
# check setting
do_restore=1
if [ "$do_restore" -eq 1 ] && [ ! -e "$RESTORE_WORK_DIR" ]; then
    echo "RESTORE_WORK_DIR is not created."
    do_restore=0
fi
if [ "$do_restore" -eq 1 ] && [ -z "$ARCHIVE_FILE_NAME" ]; then
    echo "ARCHIVE_FILE_NAME is not set."
    do_restore=0
fi
if [ "$do_restore" -eq 1 ] && [ "${#RESTORE_DIC[@]}" -eq 0 ]; then
    echo "RESTORE_DIC is not set."
    do_restore=0
fi

if [ "$do_restore" -eq 1 ]; then
    # expand the backup archive file 
    tar xvfz $ARCHIVE_FILE_PATH -C $RESTORE_WORK_DIR
    chown -R $USER:$USER $RESTORE_WORK_DIR/*

    # distribute the expanded file/directory according to the RESTORE_DIC
    IFS=' '
    for item in "${RESTORE_DIC[@]}"
    do
        set -- $item  
       dst_path=$(echo $HOME/$2 | tr -d "'")
       mkdir -p $dst_path
       src_path=$(echo $RESTORE_WORK_DIR/$1 | tr -d "'")
       echo "Placement: $src_path -> $dst_path"
       cp -rp $src_path $dst_path
    done

    # delete restore work directory
    rm -rf $RESTORE_WORK_DIR
fi

echo
echo "... finished."